# Project 1 Temp v2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

print('Import libraries successful!')

Import libraries successful!


## Preprocessing

In [2]:
# Read in the data
data = pd.read_csv('train_tweets.txt', sep="\t", header=None)
#Separate tweet into id and text
data.columns = ['id','tweet']
print(data['tweet'][1])

Going to watch Grey's on the big screen - Thursday indulgence.....


In [3]:
id_count = {}
for i in range(len(data)):
    if data['id'][i] in id_count.keys() :
        id_count[data['id'][i]] += 1
    else:
        id_count[data['id'][i]] = 1

#print(id_count[8746])
#print(id_count[2423])
#print(len(id_count.keys()))

cutoff = 40
delete_id_array = []
for i in id_count:
    if id_count[i] < cutoff:
        delete_id_array.append(i)
        

print(len(delete_id_array))
#print(delete_id_array[0])

91
35
9293
7362
2423


In [4]:
#data = data
for j in delete_id_array:  
    data = data[data.id != j]
data.head()

,id,tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...


In [5]:
print(len(data))

147712


In [6]:
# Covert tweets to lowercase
data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['tweet'].head()

0       @handle let's try and catch up live next week!
1    going to watch grey's on the big screen - thur...
2    @handle my pleasure patrick....hope you are well!
3    @handle hi there! been traveling a lot and lot...
4    rt @handle looking to drink clean & go green? ...
Name: tweet, dtype: object

## Feature Engineering

Selecting the appropriate features for the model and engineering them for training

#### Word and Character counts

In [7]:
data['word_count'] = data['tweet'].apply(lambda x: len(str(x).split(" ")))
data['char_count'] = data['tweet'].str.len() ## this also includes spaces
data[['tweet','char_count']].head()

,tweet,char_count
0,@handle let's try and catch up live next week!,46
1,going to watch grey's on the big screen - thur...,66
2,@handle my pleasure patrick....hope you are well!,49
3,@handle hi there! been traveling a lot and lot...,132
4,rt @handle looking to drink clean & go green? ...,109


#### Average Word Length

In [8]:
# Average word length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['tweet'].apply(lambda x: avg_word(x))
data[['tweet','avg_word']].head()

,tweet,avg_word
0,@handle let's try and catch up live next week!,4.222222
1,going to watch grey's on the big screen - thur...,5.090909
2,@handle my pleasure patrick....hope you are well!,6.142857
3,@handle hi there! been traveling a lot and lot...,3.925926
4,rt @handle looking to drink clean & go green? ...,4.789474


#### Stop Words

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
stop = stopwords.words('english')
data['stopwords'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
data[['tweet','stopwords']].head()

,tweet,stopwords
0,@handle let's try and catch up live next week!,2
1,going to watch grey's on the big screen - thur...,3
2,@handle my pleasure patrick....hope you are well!,3
3,@handle hi there! been traveling a lot and lot...,11
4,rt @handle looking to drink clean & go green? ...,4


#### Average length w/o stop words

In [10]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['tweet'].apply(lambda x: avg_word(x))
data[['tweet','avg_word']].head()

,tweet,avg_word
0,@handle let's try and catch up live next week!,4.222222
1,going to watch grey's on the big screen - thur...,5.090909
2,@handle my pleasure patrick....hope you are well!,6.142857
3,@handle hi there! been traveling a lot and lot...,3.925926
4,rt @handle looking to drink clean & go green? ...,4.789474


#### Hashtags, links and @'s

In [11]:
# Number of special charecters
data['hastags'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
#data[['tweet','hastags']].head()
data['ats'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('@handle')]))
#data[['tweet','ats']].head()
data['links'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('http')]))
data[['tweet','links']].head()

,tweet,links
0,@handle let's try and catch up live next week!,0
1,going to watch grey's on the big screen - thur...,0
2,@handle my pleasure patrick....hope you are well!,0
3,@handle hi there! been traveling a lot and lot...,0
4,rt @handle looking to drink clean & go green? ...,0


#### Profanities

In [12]:
from profanity import profanity 
def prof_count(sentence):
    count = 0
    words = sentence.split()
    for word in words:
        if profanity.contains_profanity(word):
            count +=1
    return (count)

def prof(sentence):
    if profanity.contains_profanity(sentence):
            return 1
    return 0

data['profanity'] = data['tweet'].apply(lambda x: prof(x))
data[['tweet','profanity']].head()

,tweet,profanity
0,@handle let's try and catch up live next week!,0
1,going to watch grey's on the big screen - thur...,0
2,@handle my pleasure patrick....hope you are well!,0
3,@handle hi there! been traveling a lot and lot...,0
4,rt @handle looking to drink clean & go green? ...,0


## Model Selection

### Define Features and split data to train/test

In [13]:
from sklearn.model_selection import train_test_split

features = ['word_count','avg_word','char_count','stopwords','hastags','ats','links','profanity']
data_small = data[:100000]
feat = ['word_count','char_count','avg_word','stopwords','hastags','numerics','upper']
# Split dataset into training set and test set
#X_train, X_test, y_train, y_test = train_test_split(data_small[features], data_small.id, test_size=0.1,random_state=42)
X_train, X_test, y_train, y_test = train_test_split(data[features], data.id, test_size=0.1,random_state=42)



### Predict with different models

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Scale the Features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


In [15]:
#Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
bayes_prediction = gnb.predict(X_test)

In [16]:
#Support Vector Machine (SVM)
#svclassifier = SVC(kernel='poly', degree=2)
#svclassifier.fit(X_train, y_train)
#SVM_prediction = svclassifier.predict(X_test)

In [17]:
#Decision Tree

DT_classifier = DecisionTreeClassifier()
DT_classifier.fit(X_train, y_train)
Decision_Tree_prediction = DT_classifier.predict(X_test)

In [18]:
# Random Forrest

#RF_classifier = RandomForestClassifier(n_estimators=20, random_state=0)
#RF_classifier.fit(X_train, y_train)
#RF_prediction = RF_classifier.predict(X_test)

In [19]:
# K-Nearest Neighbours

#KNN_classifier = KNeighborsClassifier(20)
#KNN_classifier.fit(X_train, y_train)
#KNN_prediction = KNN_classifier.predict(X_test)

In [20]:
from sklearn import metrics
print("Bayes Accuracy:     ",round(metrics.accuracy_score(y_test, bayes_prediction)*100,2), "%")
#print("SVM Accuracy:     ",round(metrics.accuracy_score(y_test, SVM_prediction)*100,2), "%")
print("D-Tree Accuracy:    ",round(metrics.accuracy_score(y_test, Decision_Tree_prediction)*100,2), "%")
#print("R-Forrest Accuracy: ",round(metrics.accuracy_score(y_test, RF_prediction)*100,2), "%")
#print("K-NN Accuracy:      ",round(metrics.accuracy_score(y_test, KNN_prediction)*100,2), "%")






Bayes Accuracy:      1.92 %
D-Tree Accuracy:     6.55 %
